In [4]:
#install needed packages
!pip install snorkel
!pip install textblob

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
#import libraries and modules
import io
import pandas as pd

In [6]:
!pip install snorkel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
#Snorkel
from snorkel.labeling import LabelingFunction
import re
from snorkel.preprocess import preprocessor
from textblob import TextBlob
from snorkel.labeling import PandasLFApplier
from snorkel.labeling.model import LabelModel
from snorkel.labeling import LFAnalysis
from snorkel.labeling import filter_unlabeled_dataframe
from snorkel.labeling import labeling_function

In [8]:
!pip install spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:

#NLP packages
import spacy
from nltk.corpus import stopwords
import string
import nltk
import nltk.tokenize
punc = string.punctuation
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
#Supervised learning
from tqdm import tqdm_notebook as tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
##Deep learning libraries and APIs
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [11]:
# store the dataset as a Pandas Dataframe
df = pd.read_csv('train_file.csv')
df = pd.DataFrame()
training_data = pd.read_csv('train_file.csv')
#conduct some data cleaning
# df = df.drop(['publish_date', 'Unnamme 'pd' is not defineded: 2'], axis=1)
# df = df.rename(columns = {'headline_text': 'text'})
# df['Title'] = df['Title'].astype(str)
#check the data info
training_data.head()
training_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55932 entries, 0 to 55931
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   IDLink             55932 non-null  object 
 1   Title              55932 non-null  object 
 2   Headline           55932 non-null  object 
 3   Source             55757 non-null  object 
 4   Topic              55932 non-null  object 
 5   PublishDate        55932 non-null  object 
 6   Facebook           55932 non-null  int64  
 7   GooglePlus         55932 non-null  int64  
 8   LinkedIn           55932 non-null  int64  
 9   SentimentTitle     55932 non-null  float64
 10  SentimentHeadline  55932 non-null  float64
dtypes: float64(2), int64(3), object(6)
memory usage: 4.7+ MB


In [12]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
stop_words = set(stopwords.words('english'))
title = list(training_data['Title'])
headline = list(training_data['Headline'])

In [14]:
import re
title_words = [re.findall(r"[\w']+", x) for x in title]
training_data['Title'] = [re.findall(r"[\w']+", x) for x in title]
headline_words = [re.findall(r"[\w']+", x) for x in headline]
print(headline_words[0])

['Obama', 'Lays', 'Wreath', 'at', 'Arlington', 'National', 'Cemetery', 'President', 'Barack', 'Obama', 'has', 'laid', 'a', 'wreath', 'at', 'the', 'Tomb', 'of', 'the', 'Unknowns', 'to', 'honor']


In [15]:
new_title_words = [];
for wordList in title_words:
    new_words = list(filter(lambda w: w not in stop_words,wordList))
    new_title_words.append(new_words)
new_headline_words = []
for wordList in headline_words:
    new_words = list(filter(lambda w: w not in stop_words,wordList))
    new_headline_words.append(new_words)

In [16]:
sentiment_titles = list(training_data['SentimentTitle'])
sentiment_headlines = list(training_data['SentimentHeadline'])

In [17]:
#define constants to represent the class labels :positive, negative, and abstain
POSITIVES = list()
NEGATIVES = list()
NEUTRALS =  list()
#define function which looks into the input words to represent a proper label
# def keyword_lookup(x, keywords, label):  
#     if any(word in x.text.lower() for word in keywords):
#         return label
#     return ABSTAIN
# #define function which assigns a correct label
# def make_keyword_lf(keywords, label=POSITIVE):
#     return LabelingFunction(
#         name=f"keyword_{keywords[0]}",
#         f=keyword_lookup,
#         resources=dict(keywords=keywords, label=label))




for x in range(len(sentiment_titles)):
    for i in new_headline_words[x]:
        if(sentiment_titles[x] < 0 ):
            NEGATIVES.append(i)
        elif(sentiment_titles[x] > 0 ):
            POSITIVES.append(i)
        elif(sentiment_titles[x] == 0 ):
            NEUTRALS.append(i)

In [18]:
#define constants to represent the class labels :positive, negative, and abstain
POSITIVES_headline = list()
NEGATIVES_headline = list()
NEUTRALS_headline =  list()
#define function which looks into the input words to represent a proper label
# def keyword_lookup(x, keywords, label):  
#     if any(word in x.text.lower() for word in keywords):
#         return label
#     return ABSTAIN
# #define function which assigns a correct label
# def make_keyword_lf(keywords, label=POSITIVE):
#     return LabelingFunction(
#         name=f"keyword_{keywords[0]}",
#         f=keyword_lookup,
#         resources=dict(keywords=keywords, label=label))




for x in range(len(sentiment_headlines)):
    for i in new_headline_words[x]:
        if(sentiment_headlines[x] < 0 ):
            NEGATIVES_headline.append(i)
        elif(sentiment_headlines[x] > 0 ):
            POSITIVES_headline.append(i)
        elif(sentiment_headlines[x] == 0 ):
            NEUTRALS_headline.append(i)

In [19]:
#set up a preprocessor function to determine polarity & subjectivity using textlob pretrained classifier 
@preprocessor(memoize=True)
def textblob_sentiment(x):
    scores = TextBlob(x.text)
    x.polarity = scores.sentiment.polarity
    x.subjectivity = scores.sentiment.subjectivity
    return x
#find polarity
@labeling_function(pre=[textblob_sentiment])
def textblob_polarity(x):
    return POSITIVES if x.polarity > 0.6 else NEUTRALS
#find subjectivity 
@labeling_function(pre=[textblob_sentiment])
def textblob_subjectivity(x):
    return POSITIVES if x.subjectivity >= 0.5 else NEUTRALS
def keyword_lookup(x, keywords, label):  
    if any(word in x.text.lower() for word in keywords):
        return label
    return "NEUTRAL"
def make_keyword_lf(keywords, label="POSITIVE"):
    return LabelingFunction(
        name=f"keyword_{keywords[0]}",
        f=keyword_lookup,
        resources=dict(keywords=keywords, label=label))
#combine all the labeling functions 
lfs = [make_keyword_lf(POSITIVES_headline), make_keyword_lf(NEGATIVES_headline), textblob_polarity, textblob_subjectivity]
#apply the lfs on the dataframe
applier = PandasLFApplier(lfs=lfs)
L_snorkel = applier.apply(df=df)

/usr/local/lib/python3.9/dist-packages/tqdm/std.py:766: RuntimeWarning: divide by zero encountered in long_scalars
  total = df.size // df.shape[axis]
1it [00:00, 1526.87it/s]


In [20]:
new_title_words_df = pd.DataFrame(new_title_words)
new_headline_words_df = pd.DataFrame(new_headline_words)
new_headline_words_df

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,Obama,Lays,Wreath,Arlington,National,Cemetery,President,Barack,Obama,laid,...,None,None,None,None,None,None,None,None,None,None
1,Tim,Haywood,investment,director,business,unit,head,fixed,income,Gam,...,None,None,None,None,None,None,None,None,None,None
2,Nouriel,Roubini,NYU,professor,chairman,Roubini,Global,Economics,explains,global,...,None,None,None,None,None,None,None,None,None,None
3,Finland's,economy,expanded,marginally,three,months,ended,December,contracting,previous,...,None,None,None,None,None,None,None,None,None,None
4,Tourism,public,spending,continued,boost,economy,January,light,contraction,private,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55927,Retired,Cuban,leader,Fidel,Castro,slammed,President,Barack,Obama's,recent,...,None,None,None,None,None,None,None,None,None,None
55928,President,Obama,caught,predictable,flak,political,right,way,reacted,react,...,None,None,None,None,None,None,None,None,None,None
55929,While,Trump,wants,put,large,tariffs,imports,major,trading,partners,...,None,None,None,None,None,None,None,None,None,None
55930,Microsoft,business,customers,finally,beginning,take,delivery,giant,Windows,10,...,None,None,None,None,None,None,None,None,None,None


In [21]:
new_title_words_df = new_title_words_df.fillna('')
new_headline_words_df = new_headline_words_df.fillna('')
new_headline_words_df

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,Obama,Lays,Wreath,Arlington,National,Cemetery,President,Barack,Obama,laid,...,,,,,,,,,,
1,Tim,Haywood,investment,director,business,unit,head,fixed,income,Gam,...,,,,,,,,,,
2,Nouriel,Roubini,NYU,professor,chairman,Roubini,Global,Economics,explains,global,...,,,,,,,,,,
3,Finland's,economy,expanded,marginally,three,months,ended,December,contracting,previous,...,,,,,,,,,,
4,Tourism,public,spending,continued,boost,economy,January,light,contraction,private,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55927,Retired,Cuban,leader,Fidel,Castro,slammed,President,Barack,Obama's,recent,...,,,,,,,,,,
55928,President,Obama,caught,predictable,flak,political,right,way,reacted,react,...,,,,,,,,,,
55929,While,Trump,wants,put,large,tariffs,imports,major,trading,partners,...,,,,,,,,,,
55930,Microsoft,business,customers,finally,beginning,take,delivery,giant,Windows,10,...,,,,,,,,,,


In [22]:
tfidf_vect = TfidfVectorizer()
new_title_words_df
# new_title_words_df = new_title_words_df.apply(lambda text: " ".join(set(text)),axis=1)
new_title_words_df['Title'] = new_title_words_df[new_title_words_df.columns[1:]].apply(
    lambda x: ' '.join(x.dropna().astype(str)),
    axis=1
)

# # new_title_words_df

In [23]:
new_title_words_df = new_title_words_df.iloc[:,25:]


In [24]:
new_title_words_df

,Title
0,Lays Wreath Arlington National Cemetery ...
1,Look Health Chinese Economy
2,Roubini Global Economy Not Back 2008 ...
3,GDP Expands In Q4
4,govt spending buoys Thai economy January ...
...,...
55927,Castro Lashes Out Obama After Cuba Visit ...
55928,CRISP Obama's strategic reaction Brussels ...
55929,Trump's 45 Percent Tariffs Are Bad Try Obama's...
55930,finally releases giant Surface ...


In [25]:
# tfidf_vect = TfidfVectorizer()
# new_headline_words_df
# new_title_words_df = new_title_words_df.apply(lambda text: " ".join(set(text)),axis=1)
new_headline_words_df['Headline'] = new_headline_words_df[new_headline_words_df.columns[1:]].apply(
    lambda x: ' '.join(x.dropna().astype(str)),
    axis=1
)
# new_headline_words_df = new_headline_words_df.iloc[:,61:]

# # print((new_headline_words_df))
# # new_title_words_df

In [26]:
new_headline_words_df = new_headline_words_df.iloc[:,61:]


In [27]:
new_headline_words_df

,Headline
0,Lays Wreath Arlington National Cemetery Presid...
1,Haywood investment director business unit head...
2,Roubini NYU professor chairman Roubini Global ...
3,economy expanded marginally three months ended...
4,public spending continued boost economy Januar...
...,...
55927,Cuban leader Fidel Castro slammed President Ba...
55928,Obama caught predictable flak political right ...
55929,Trump wants put large tariffs imports major tr...
55930,business customers finally beginning take deli...


In [36]:
join=pd.DataFrame()
join['title_headline']=new_title_words_df['Title']+new_headline_words_df['Headline']

In [ ]:
# import re
# s = "The   fox jumped   over    the log."
for sentence in join['title_headline']:
  re.sub("\s\s+" , " ", sentence)
  print(sentence)
# for word in join.iloc[2]:
#   print(word)

In [38]:
for word in join.iloc[2]:
  print(word)

Roubini Global Economy Not Back 2008                  Roubini NYU professor chairman Roubini Global Economics explains global economy facing conditions                                                


In [27]:
# df["Period"] = df[["Title", "Headline"]].apply("-".join, axis=1)

In [42]:
# X_tfidf = tfidf_vect.fit_transform(join['title_headline'])
# # print(X_tfidf.shape)
# X_tfidf
# print(tfidf_vect.get_feature_names())
# X_tfidf = pd.DataFrame(X_tfidf.toarray())
# X_tfidf

<55932x41176 sparse matrix of type '<class 'numpy.float64'>'
	with 1102023 stored elements in Compressed Sparse Row format>

In [43]:
title_headline_labels = list()
for i in range(len(sentiment_headlines)):
    if(sentiment_headlines[i]+sentiment_titles[i] < 0):
        title_headline_labels.append(-1)
    elif(sentiment_headlines[i]+sentiment_titles[i] > 0):
        title_headline_labels.append(1)
    else:
        title_headline_labels.append(0)
pd.DataFrame(title_headline_labels)
sentiment_title_headlines_df = pd.DataFrame(title_headline_labels)

In [44]:
sentiment_title_headlines_df.shape

(55932, 1)

In [45]:
from sklearn.feature_extraction.text import CountVectorizer

In [46]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(join['title_headline'])
# X_test = vectorizer.transform(join['title_headline'])
# X= X_tfidf
y = sentiment_title_headlines_df


In [47]:
X

<55932x41176 sparse matrix of type '<class 'numpy.int64'>'
	with 1102023 stored elements in Compressed Sparse Row format>

In [48]:
y

,0
0,-1
1,1
2,-1
3,1
4,1
...,...
55927,-1
55928,1
55929,1
55930,-1


In [49]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)


In [50]:
#fit Log Regression Model
clf= LogisticRegression()
clf.fit(X_train,y_train)
clf.score(X_test,y_test)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

/usr/local/lib/python3.9/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


              precision    recall  f1-score   support

          -1       0.79      0.81      0.80      6289
           0       0.39      0.07      0.11       104
           1       0.74      0.73      0.74      4794

    accuracy                           0.77     11187
   macro avg       0.64      0.54      0.55     11187
weighted avg       0.77      0.77      0.77     11187



/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [1]:
# X_tfidf = tfidf_vect.fit_transform(new_headline_words_df['Headline'])
# print(X_tfidf.shape)
# print(tfidf_vect.get_feature_names())
# X_tfidf = pd.DataFrame(X_tfidf.toarray())
# X_tfidf

NameError: ignored

In [ ]:
# headline_labels = list()
# for i in range(len(sentiment_headlines)):
#     if(sentiment_headlines[i] < 0):
#         headline_labels.append(-1)
#     elif(sentiment_headlines[i] > 0):
#         headline_labels.append(1)
#     else:
#         headline_labels.append(0)
# pd.DataFrame(headline_labels)
# sentiment_headlines_df = pd.DataFrame(headline_labels)

In [ ]:
# sentiment_titles_df.shape

In [ ]:
# sentiment_headlines_df.shape

In [ ]:
# X= X_tfidf
# y = sentiment_titles_df
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
# #fit Log Regression Model
# clf= LogisticRegression()
# clf.fit(X_train,y_train)
# clf.score(X_test,y_test)
# y_pred = clf.predict(X_test)
# print(classification_report(y_test, y_pred))

In [ ]:
# X= X_tfidf
# y = sentiment_headlines_df
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
# #fit Log Regression Model
# clf= LogisticRegression()
# clf.fit(X_train,y_train)
# clf.score(X_test,y_test)
# y_pred = clf.predict(X_test)
# print(classification_report(y_test, y_pred))